In [51]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [74]:
def get_rsi(data, period=14):
   
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # Prevent division by zero
    rs = avg_gain / avg_loss.replace(0, np.nan)  # Replace 0 with NaN to avoid division by zero
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

In [75]:
def get_macd(data, short_window=12, long_window=26, signal_window=9):

    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    histogram = macd - signal
    return macd, signal, histogram

In [76]:
def get_bollinger_bands(data, window=20, num_std=2):
   
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    bandwidth = (upper_band - lower_band) / rolling_mean
    return upper_band, lower_band, bandwidth


In [56]:
def get_momentum(data, period=14):
 
    return data['Close'].diff(period)


In [57]:
def get_volatility(data, period=14):
  
    return data['Close'].pct_change().rolling(window=period).std()


In [59]:
def get_stock_data(ticker):
    
    data = yf.download(ticker, period="1y")  # Download 1 year of data
    return data

In [79]:
def prepare_data(data):
    """
    Prepares the data for machine learning by adding features and labels.
    """

    # Calculate price changes FIRST
    data['Price_Change'] = data['Close'].pct_change()

    # Add RSI
    data['RSI'] = get_rsi(data)

    # Check if RSI was created successfully (case insensitive)
    rsi_column_names = ['RSI', 'rsi', 'Rsi']
    if not any(col in data.columns for col in rsi_column_names):
        raise ValueError("RSI column not created. Check get_rsi() function.")

    # Add Moving Averages
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()

    # Add MACD
    macd, signal, histogram = get_macd(data)
    data['MACD'] = macd
    data['MACD_Signal'] = signal
    data['MACD_Histogram'] = histogram

    # Add Bollinger Bands
    upper_band, lower_band, bandwidth = get_bollinger_bands(data)
    data['Bollinger_Upper'] = upper_band
    data['Bollinger_Lower'] = lower_band
    data['Bollinger_Bandwidth'] = bandwidth

    # Add Momentum
    data['Momentum'] = get_momentum(data)

    # Add Volatility
    data['Volatility'] = get_volatility(data)

    # Add Volume Change
    data['Volume_Change'] = data['Volume'].pct_change().fillna(0)

    # Define buy/sell signals with NaN handling
    data['Signal'] = 0  # Ensure indentation is correct

    buy_condition = (data['RSI'] < 30) & (data['Price_Change'] > 0)
    buy_condition.fillna(False, inplace=True)  # Fix NaN handling
    data.loc[buy_condition, 'Signal'] = 1  # Buy signal

    sell_condition = (data['RSI'] > 70) & (data['Price_Change'] < 0)
    sell_condition.fillna(False, inplace=True)  # Fix NaN handling
    data.loc[sell_condition, 'Signal'] = -1  # Sell signal

    # Drop rows with NaN values (due to feature calculations)
    data.dropna(inplace=True)

    return data



In [80]:
def train_model(data):
 
    # Features
    features = ['RSI', 'SMA_10', 'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram',
                'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Bandwidth',
                'Momentum', 'Volatility', 'Volume_Change']
    X = data[features]
    # Target
    y = data['Signal']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the Logistic Regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy:.2f}")

    return model

In [81]:
def generate_alerts(data, model):
   
    # Features
    features = ['RSI', 'SMA_10', 'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram',
                'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Bandwidth',
                'Momentum', 'Volatility', 'Volume_Change']
    # Predict signals using the trained model
    data['Predicted_Signal'] = model.predict(data[features])

    # Generate alerts
    alerts = []
    for i in range(len(data)):
        if data['Predicted_Signal'][i] == 1:
            alerts.append(("Buy", data.index[i], data['Close'][i]))
        elif data['Predicted_Signal'][i] == -1:
            alerts.append(("Sell", data.index[i], data['Close'][i]))
        else:
            alerts.append(("Hold", data.index[i], data['Close'][i]))

    return alerts

In [82]:
def analyze_stock(ticker):
    """
    Analyzes a stock and provides buy/sell/hold alerts.
    """
    # Fetch and prepare data
    data = get_stock_data(ticker)
    data = prepare_data(data)

    # Train the model
    model = train_model(data)

    # Generate alerts
    alerts = generate_alerts(data, model)

    # Print alerts
    print(f"\nAlerts for {ticker}:")
    for alert in alerts:
        print(f"{alert[0]} Signal at {alert[1]}: Price: {alert[2]}")

if __name__ == "__main__":
    # Input stock ticker from the user
    ticker = input("Enter the stock ticker symbol (e.g., AAPL, MSFT, TSLA): ").strip().upper()

    # Analyze the stock
    analyze_stock(ticker)

Enter the stock ticker symbol (e.g., AAPL, MSFT, TSLA):  AAPL


[*********************100%***********************]  1 of 1 completed
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/var/folders/yj/l9hmqz8x4ml53637xyb0vkyr0000gn/T/ipykernel_14687/396016468.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data['Predicted_Signal'][i] == 1:
/var/folders/yj/l9hmqz8x4ml

Model Accuracy: 0.80


KeyError: 0